In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
conn = sqlite3.connect('MatchDB.db')
df = pd.read_sql_query("SELECT * FROM Expected_Games", conn)

In [3]:
df = df.drop(['HomeTeam', 'AwayTeam', 'League', 'Country', 'date', 'id', 'match_id', 'HGames', 'AGames'], axis=1)
df = df.drop(['xAwayPoss', 'xAwayShots', 'xAwaySonT', 'xAwaySoffT', 'xAwayBS', 'xAwayCor', 'xAwayOff', 'xAwayFoul', 'xAwayYellow', 'xAwayPass', 'xAwayAccPass', 'xAwayPassOff', 'xAwayAccLongB', 'xAwayAccLongBpercent', 'xAwayAccCross', 'xAwayAccCrosspercent', 'xAwaySuccDribb', 'xAwaySuccDribbpercent', 'xAwayDuelsW', 'xAwayTackW', 'xAwayTackWpercent', 'xAwayInt', 'xAwayClear'], axis= 1)
df = df.dropna()

In [4]:
prediction = df.drop(['HGoals', 'AGoals'], axis=1).astype(float)
target_1 = df['HGoals'].astype(float)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(prediction, target_1, train_size=0.2, random_state=42)

In [6]:
my_pipe = Pipeline ([
        ('imputer', SimpleImputer(strategy='median')),
        ('std_scaler', StandardScaler()),
])
# X_train = my_pipe.fit_transform(X_train)

In [7]:
param_grid = [{'n_estimators': [2, 5, 10, 100]}]

forest_reg = RandomForestRegressor(random_state=42)

grid_search = GridSearchCV(forest_reg, param_grid, cv=3,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

grid_search.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid=[{'n_estimators': [2, 5, 10, 100]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [8]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.182425,0.008305,0.002327,0.000471,2,{'n_estimators': 2},-2.331594,-2.318155,-2.212732,-2.287494,0.053149,4,-0.629640,-0.629023,-0.654683,-0.637782,0.011953
1,0.445738,0.008276,0.002983,0.000014,5,{'n_estimators': 5},-1.830261,-1.889629,-1.761763,-1.827218,0.052245,3,-0.384559,-0.384448,-0.395407,-0.388138,0.005140
2,0.878147,0.014334,0.004984,0.000002,10,{'n_estimators': 10},-1.739884,-1.736827,-1.662401,-1.713038,0.035827,2,-0.302909,-0.288658,-0.287504,-0.293023,0.007006
3,9.020265,0.093911,0.033568,0.000949,100,{'n_estimators': 100},-1.550578,-1.583020,-1.481969,-1.538522,0.042125,1,-0.211415,-0.213373,-0.221068,-0.215285,0.004166


In [9]:
grid_search.best_params_

{'n_estimators': 100}

In [10]:
# X_test = my_pipe.transform(X_test)

In [11]:
prediction = grid_search.predict(X_test)

In [12]:
mse = mean_squared_error(y_test, prediction)
rmse = np.sqrt(mse)
rmse

1.2262841048247768

In [13]:
mean_value = df["HGoals"].mean()
mean_value

1.4815588030619347

In [14]:
rmse/ mean_value

0.8276985714575876

In [15]:
game = 333
print(prediction[game])
print(y_test[game])
print(X_test[game])

2.07
0.0


KeyError: 333

In [16]:
print(X_test)

       xHomePoss  xHomeShots  xHomeSonT  xHomeSoffT  xHomeBS  xHomeCor  \
6421       52.22        9.25       3.33        5.92     2.22      6.61   
22826      38.30        9.57       3.69        5.88     2.50      2.89   
5784       60.05       14.97       2.88       12.09     4.16      5.59   
24383      61.37       14.72       5.28        9.44     3.33      5.96   
3486       47.00       11.52       4.09        7.43     2.64      5.77   
...          ...         ...        ...         ...      ...       ...   
16061      45.08        9.00       3.37        5.63     3.07      5.78   
19497      57.11       12.97       5.46        7.51     3.42      3.96   
3303       57.12       12.71       3.94        8.77     3.17      5.53   
15549      49.86       19.22       6.99       12.23     4.05      7.02   
18074      40.49        9.95       3.66        6.29     2.33      4.81   

       xHomeOff  xHomeFoul  xHomeYellow  xHomeRed  ...  xHomeAccLongBpercent  \
6421       3.68      12.03     